In [5]:
import torch
a = torch.tensor([1,2]).view(1,2)

In [6]:
a.shape

torch.Size([1, 2])

In [8]:
b = torch.mm(torch.tensor([1,1]).view(-1,1), a)

In [9]:
b

tensor([[1, 2],
        [1, 2]])

In [30]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
sigma = 1
man = dm.manifold.Landmarks(2,3)
scal = dm.deformationmodules.Scaling(man, sigma)

In [60]:
scal()

tensor([[ 1.0000,  0.0000],
        [-0.5000,  0.8660],
        [-0.5000, -0.8660]], grad_fn=<MulBackward0>)

In [61]:
scal.z()

tensor([[0., 0.]], grad_fn=<ViewBackward>)

In [62]:
b = torch.mm(torch.tensor([1,1]).view(-1,1), scal.z().long())

In [63]:
b

tensor([[0, 0],
        [0, 0]])

In [64]:
scal.z()

tensor([[0., 0.]], grad_fn=<ViewBackward>)

In [69]:
x = scal.scal_vec + scal.z()

In [67]:
torch.ones(scal.scal_vec.shape)*scal.z()

In [70]:
x

tensor([[ 1.0000,  0.0000],
        [-0.5000,  0.8660],
        [-0.5000, -0.8660]], grad_fn=<AddBackward0>)

In [72]:
scal.z() * torch.ones(x.shape)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<MulBackward0>)

In [82]:
f, s, m = scal.field_generator()

In [78]:
points = torch.rand(5,2)
print(points)

tensor([[0.5348, 0.0273],
        [0.2653, 0.3726],
        [0.9796, 0.9171],
        [0.5800, 0.8532],
        [0.4814, 0.7697]])


In [89]:
dm.structuredfield.StructuredField_0(s, m, sigma)(points)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<AsStridedBackward>)

In [88]:
print('support:', s)
print('moments:', m)

support: tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<MulBackward0>)
moments: tensor([[ 1.0000,  0.0000],
        [-0.5000,  0.8660],
        [-0.5000, -0.8660]], grad_fn=<MulBackward0>)


In [94]:
from defmod.kernels import gauss_kernel, rel_differences, K_xy
k=0
ker_vec = gauss_kernel(rel_differences(points, s), k, sigma)


In [95]:
ker_vec

tensor([0.8664, 0.8664, 0.8664, 0.9007, 0.9007, 0.9007, 0.4064, 0.4064, 0.4064,
        0.5873, 0.5873, 0.5873, 0.6623, 0.6623, 0.6623], grad_fn=<ExpBackward>)

In [97]:
ker_vec = ker_vec.reshape((points.shape[0], s.shape[0]) + tuple(ker_vec.shape[1:]))


In [98]:
ker_vec

tensor([[0.8664, 0.8664, 0.8664],
        [0.9007, 0.9007, 0.9007],
        [0.4064, 0.4064, 0.4064],
        [0.5873, 0.5873, 0.5873],
        [0.6623, 0.6623, 0.6623]], grad_fn=<AsStridedBackward>)

In [102]:
tmp1 = torch.tensordot(torch.eye(2), ker_vec, dims=0)
tmp2 = torch.transpose(tmp1, 0, 2)

torch.tensordot(tmp2, m, dims=([2, 3], [1, 0]))

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<AsStridedBackward>)

In [107]:
tmp2.shape, m.shape

(torch.Size([5, 2, 2, 3]), torch.Size([3, 2]))

In [108]:
tmp2

tensor([[[[0.8664, 0.8664, 0.8664],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.8664, 0.8664, 0.8664]]],


        [[[0.9007, 0.9007, 0.9007],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.9007, 0.9007, 0.9007]]],


        [[[0.4064, 0.4064, 0.4064],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.4064, 0.4064, 0.4064]]],


        [[[0.5873, 0.5873, 0.5873],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.5873, 0.5873, 0.5873]]],


        [[[0.6623, 0.6623, 0.6623],
          [0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000],
          [0.6623, 0.6623, 0.6623]]]], grad_fn=<TransposeBackward0>)